In [1]:
import openai
import os
from PyPDF2 import PdfReader

openai.api_key = ''

In [2]:
def get_completion(prompt, model="gpt-4", temperature=0): 
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, 
    )
    return response.choices[0].message["content"]

In [3]:
def get_num_pages_from_pdf(filename):
    # creating a pdf reader object
    reader = PdfReader(filename)
    return len(reader.pages)

In [4]:
def text_from_pdf(filename, page_to_extract=-1, max_words=4097):
    # creating a pdf reader object
    reader = PdfReader(filename)
    
    text = ''
    if (page_to_extract == -1):
        for page in reader.pages:
            
            # extracting text from page
            text_tmp = page.extract_text()
            text_tmp = text_tmp.replace("\n", "")
            text += text_tmp
    else:
        if (page_to_extract > len(reader.pages)):
            return -1
        
        page = reader.pages[page_to_extract]
        # extracting text from page
        text_tmp = page.extract_text()
        text_tmp = text_tmp.replace("\n", "")
        text += text_tmp

    # Split the content into words.
    words = text.split(" ")

    # Limit the number of words.
    if len(words) > max_words:
        words = words[:max_words]

    # Return the extracted text.
    return ' '.join(words)

In [19]:
max_dim = 1024
max_sentences = range(5)
temperature = 0
filename = '/Users/stjames/Dropbox/Pablo/data/gpt-pdf-summarizer/text/BIA ENG.pdf'

num_pages = get_num_pages_from_pdf(filename)
text400 = ''  # initialize an empty string to store all responses

for p in range(0, num_pages):
    text = text_from_pdf(filename, page_to_extract=p, max_words=max_dim)

    prompt = f"""
        You are a highly intelligent law student studying for the final exam of your Immigration Law Removal Defense course. Your intention is to create flashcards to study. List all of the most important aspects of the text delimited by triple backticks in question and answer format. Each question and answer pair must be about a topic fundamental to the field of removal defense. Write more questions if you feel it's important for comprehension, and fewer if not. Use Q1: A1: etc format. I need to be able to upload to Anki.
        ```{text}```
        """

    response = get_completion(prompt)
    text400 += response  # add the response to the cumulative responses string

    print ('Page ' + str(p))

print('All responses:')
print(text400)


Page 0
Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Page 13
Page 14
Page 15
Page 16
Page 17
Page 18
Page 19
Page 20
Page 21
Page 22
Page 23
Page 24
All responses:
Q1: What is the purpose of the guide prepared by the Florence Immigrant & Refugee Rights Project?
A1: The guide was prepared to assist immigrant detainees in Arizona who are representing themselves pro se in their removal proceedings.

Q2: Who is the intended audience of this guide?
A2: The intended audience of this guide are immigrant detainees in Arizona who are representing themselves in their removal proceedings.

Q3: Is this guide a substitute for legal counsel?
A3: No, this guide is not intended to provide legal advice or serve as a substitute for legal counsel.

Q4: Does the Florence Project charge for its services to immigrant detainees in Arizona?
A4: No, the Florence Project is a nonprofit legal services organization and does not charge for its services to immigrant detainee

In [17]:
text400

'Q1: What is the Florence Immigrant & Refugee Rights Project?\nA1: The Florence Immigrant & Refugee Rights Project is a nonprofit legal services organization that works with adults and children in immigration custody in Arizona.\n\nQ2: What is the purpose of the guide prepared by the Florence Project?\nA2: The guide is prepared for immigrant detainees who represent themselves in their removal proceedings.\n\nQ3: Does the Florence Project charge for their services?\nA3: No, the Florence Project does not charge for their services.\n\nQ4: Can the guide prepared by the Florence Project be shared and distributed?\nA4: Yes, the guide is copyright protected but can be shared and distributed widely to help immigrants around the country.\n\nQ5: What should you do if you adapt the information in the guide into your own publication?\nA5: If you adapt the information in the guide into your own publication, you should credit the Florence Project.\n\nQ6: Is the guide intended to provide legal advice

In [20]:
import pandas as pd
import re

def text_to_dataframe(text):
    # This pattern matches a question and its corresponding answer
    pattern = r'(Q\d+: .*?)(?=Q\d+:|$)'
    
    pairs = re.findall(pattern, text, re.DOTALL)  # re.DOTALL makes . match newline

    questions = []
    answers = []

    for pair in pairs:
        q, a = pair.split('\nA')
        questions.append(q)
        answers.append('A'+a)

    df = pd.DataFrame({
        'Front': questions,
        'Back': answers
    })
    
    return df

text = text400

df = text_to_dataframe(text)
df.to_csv('/Users/stjames/Dropbox/Pablo/data/gpt-pdf-summarizer/BIA`_flashcards.csv', index=False)


In [26]:
range(5)

range(0, 5)

In [42]:
response

"Q1: What is the categorical approach in the context of immigration law?\nA1: The categorical approach is a legal method used in immigration law to determine whether a non-citizen's conviction qualifies as a deportable offense. It involves comparing the legal definition of the crime of conviction with a federal immigration law definition.\n\nQ2: What is discussed in Chapter 5 of the Immigrant Legal Resource Center's guide?\nA2: Chapter 5 discusses issues of establishing deportability due to a conviction. It likely covers how to determine if a conviction meets the criteria for a deportable offense under immigration law.\n\nQ3: What is the focus of Chapter 9 in the Immigrant Legal Resource Center's guide?\nA3: Chapter 9 is focused on the discussion of relief. This likely refers to the various forms of relief or defenses available to immigrants facing removal proceedings.\n\nQ4: What is the purpose of the Immigrant Legal Resource Center's guide?\nA4: The guide is intended to provide compr